<a href="https://colab.research.google.com/github/axjasf/YNAB-Categorizer/blob/main/AmazonOrders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Libraries

In [ ]:
# Installing libraries
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [6]:
# Importing libraries
import pandas as pd
import numpy as np
import os
from ipywidgets import widgets
from IPython.display import display

## Global Variables

In [ ]:
### CONSTANTS ###

# Major categories
category= {
    "grocery" : "Food, Sports, Entertainment:Food - Groceries & Errands"
}

## Files

In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

HOME_PATH = "/content/drive/MyDrive/Colab Notebooks/YNAB/"

Mounted at /content/drive


In [15]:
### Config Files ###

CONFIG_PATH = HOME_PATH + "/config/"

# Read Grocery keywords
grocery_keywords_df = pd.read_csv(CONFIG_PATH + 'grocery_keywords.csv')
grocery_keywords = grocery_keywords_df.iloc[:, 0].str.lower().tolist()

# Checking, if data is valid
# grocery_keywords_df.head()

In [ ]:
### Amazon Orders ###

TRANSACTIONS_PATH = HOME_PATH + "/transactions/"

# Read Amazon Order Headers and Items
amazon_order_headers_df = pd.read_csv(TRANSACTIONS_PATH + 'amazon_order_headers.csv')
amazon_order_items_df = pd.read_csv(TRANSACTIONS_PATH + 'amazon_order_items.csv')

# Checking, if data is valid
#amazon_order_headers_df.head()
#amazon_order_items_df.head()

# PRICING

In [ ]:
### PRICE TOTALS CALCULATIONS ###
### HEADER ###

# Extract the individual payments for one row and put them into columns
def extract_individual_payments_into_columns(df, column_name):
    # Extract dollar values from the specified column
    payments_dollar_values = df[column_name].str.findall(r'\$(\d+\.\d+)')

    # Determine the maximum number of payments in any row
    max_payments = payments_dollar_values.apply(len).max()

    # Pad the lists to ensure they all have a length of max_payments
    padded_payments = payments_dollar_values.apply(lambda x: x + [None] * (max_payments - len(x)))

    # Convert the padded lists to a DataFrame with dynamic column names
    payment_columns = [f'payment_{i+1}' for i in range(max_payments)]
    df[payment_columns] = pd.DataFrame(padded_payments.tolist(), index=df.index)

    # Convert string values to float for further analysis if needed
    for col in payment_columns:
        df[col] = df[col].astype(float)

# Apply the function to the original dataframe
extract_individual_payments_into_columns(amazon_order_headers, "payments")

# amazon_order_headers.head()

In [ ]:
### PRICE TOTALS CALCULATIONS ###
### ITEMS ###

# Item Totals (Quantity * Price) #
amazon_order_items = amazon_order_items.copy()
if amazon_order_items['price'].dtype == 'object':
    amazon_order_items['price'] = pd.to_numeric(amazon_order_items['price'].str.replace('[\$,]', '', regex=True), errors='coerce')
amazon_order_items['quantity'] = amazon_order_items['quantity'].replace([' ', ''], np.nan)
amazon_order_items['quantity'] = pd.to_numeric(amazon_order_items['quantity'], errors='coerce').fillna(1)
amazon_order_items['total'] = amazon_order_items['price'] * amazon_order_items['quantity']

### Distribution of taxes etc. to individual order items ###

# Group the items dataframe by 'order id' and compute the sum of the 'total' column for each order
# Merge the summed items total with the headers dataframe on 'order id'
# Convert 'total_header' column to numeric format, setting non-numeric values to NaN
# Compute the difference between the 'total' from the headers dataframe and the computed sum of items for each order
# Compute the count of items for each order
# Merge the count of items with the merged dataframe
# Compute the 'overall' value for each order
# Merge the 'overall' value with the original items dataframe to create the 'overall' column

items_sum = amazon_order_items.groupby('order id')['total'].sum().reset_index()
merged_df = amazon_order_headers.merge(items_sum, on='order id', how='left', suffixes=('_header', '_items'))
merged_df['total_header'] = pd.to_numeric(merged_df['total_header'].str.replace('[\$,]', '', regex=True), errors='coerce')
merged_df['difference'] = merged_df['total_header'] - merged_df['total_items']
items_count = amazon_order_items.groupby('order id').size().reset_index(name='count')
merged_df = merged_df.merge(items_count, on='order id', how='left')
merged_df['residual'] = merged_df['difference'] / merged_df['count']
amazon_order_items = amazon_order_items.merge(merged_df[['order id', 'residual']], on='order id', how='left')
amazon_order_items['grand_total'] = amazon_order_items['total'] + amazon_order_items['residual'].round(2)

amazon_order_items.head()

AttributeError: ignored

# CATEGORIZATION

In [ ]:
# Initialize a category column with default value as 'non-grocery'
amazon_order_items['category'] = 'n/a'

# Check each item description against the keywords
for keyword in grocery_keywords:
    amazon_order_items.loc[amazon_order_items['description'].str.lower().str.contains(keyword, na=False), 'category'] = 'grocery'

amazon_order_items.head()


,order id,order date,quantity,description,price,Unnamed: 5,total,residual,grand_total,category
0,D01-6286295-4630651,7/2/2023,1.0,"Other Minds: The Octopus, the Sea, and the Dee...",15.49,NaN,15.49,-15.490000,0.00,n/a
1,112-3542613-6514648,8/15/2023,1.0,Nice 'N Clean Adult Flushable Wipes (8 x 42 Co...,15.99,NaN,15.99,1.380000,17.37,n/a
2,114-9993434-2529000,8/14/2023,1.0,"Mushroom White Organic, 0.5lb",3.23,0.311719,3.23,0.311719,3.54,grocery
3,114-9993434-2529000,8/14/2023,1.0,Organic Mixed Variety Heirloom Tomatoes,7.73,0.311719,7.73,0.311719,8.04,grocery
4,114-9993434-2529000,8/14/2023,1.0,Organic Banana,5.05,0.311719,5.05,0.311719,5.36,grocery


# TRANSACTIONAL CREATION

In [ ]:
### SPLIT TRANSACTION ###

def create_split(category, memo, amount):
    return (category, memo, amount)

def create_transaction(date, total_amount, payee, memo, category=None, splits=None):
    return {
        "date": date,
        "total_amount": total_amount,
        "payee": payee,
        "memo": memo,
        "category": category,
        "splits": splits
    }

def export_qif_transactions(filename, transactions):
    qif_content = "!Type:Cash\n"

    for transaction in transactions:
        # Add the transaction details
        qif_content += f"D{transaction['date']}\n"
        qif_content += f"T{transaction['total_amount']}\n"
        qif_content += f"P{transaction['payee']}\n"
        qif_content += f"M{transaction['memo']}\n"

        # If a category is provided for a simple transaction, add it
        if transaction.get('category') and not transaction.get('splits'):
            qif_content += f"L{transaction['category']}\n"

        # If splits exist for the transaction, add them
        splits = transaction.get('splits')
        if splits:
            qif_content += "LSplit Transaction\n"
            for split in splits:
                category, memo, amount = split
                qif_content += f"S{category}\n"
                qif_content += f"E{memo}\n"
                qif_content += f"${amount}\n"

        # End the transaction
        qif_content += "^\n"

    # Export to a file in Google Drive
    with open(HOME_PATH + "/results/" + filename, "w") as file:
        file.write(qif_content)

def export_qif_transactions_by_chunk(transactions, x, base_filename="transactions"):
    """
    Export the transactions in QIF format, splitting them into separate files,
    each containing up to x transactions.

    Parameters:
    - transactions: List of transaction dictionaries.
    - x: Number of transactions per file.
    - base_filename: Base name for the output files. Files will be named as base_filename_1.qif, base_filename_2.qif, ...

    Returns:
    - List of saved file paths.
    """

    # Validate x
    if not (1 <= x <= 100):
        raise ValueError("x should be between 1 and 100")

    saved_files = []

    for idx, i in enumerate(range(0, len(transactions), x)):
        chunk = transactions[i:i+x]
        qif_content = "!Type:Cash\n"

        for transaction in chunk:
            # Add the transaction details
            qif_content += f"D{transaction['date']}\n"
            qif_content += f"T{transaction['total_amount']}\n"
            qif_content += f"P{transaction['payee']}\n"
            qif_content += f"M{transaction['memo']}\n"

            # If a category is provided for a simple transaction, add it
            if transaction.get('category') and not transaction.get('splits'):
                qif_content += f"L{transaction['category']}\n"

            # If splits exist for the transaction, add them
            splits = transaction.get('splits')
            if splits:
                qif_content += "LSplit Transaction\n"
                for split in splits:
                    category, memo, amount = split
                    qif_content += f"S{category}\n"
                    qif_content += f"E{memo}\n"
                    qif_content += f"${amount}\n"

            # End the transaction
            qif_content += "^\n"

        # Define filename for this chunk
        filename = f"{base_filename}_{idx+1}.qif"
        filepath = HOME_PATH + "/results/" + filename

        # Export to a file
        with open(filepath, "w") as file:
            file.write(qif_content)

        saved_files.append(filepath)

    return saved_files


In [ ]:
# Using the provided functions to regenerate the QIF transactions and export them

# Generate the QIF transactions

transactions = []

from datetime import datetime

# Iterate over each unique order
for order_id, group in amazon_order_items.groupby('order id'):
    order_date = group['order date'].iloc[0]
    total_amount = group['grand_total'].sum()
    payee = "Amazon" + " " + order_id

    if len(group) == 1: # Check for single item transactions
        row = group.iloc[0]
        if row['category'] == 'grocery':
            transaction = create_transaction(order_date, f"{row['grand_total']:.2f}", "Amazon " + order_id, "", category=category["grocery"])
        else:
            transaction = create_transaction(order_date, f"{total_amount:.2f}", payee, memo=row['description'])
        transactions.append(transaction)

    else:
        # For multiple items in the transaction:
        # Grocery split amount
        grocery_total = group.loc[group['category'] == 'grocery', 'grand_total'].sum()

        # Non-grocery splits
        non_grocery_splits = []
        for _, row in group[group['category'] != 'grocery'].iterrows():
          # Ensure row has valid data
          if 'description' in row and 'grand_total' in row:
            split = create_split("", row['description'], f"{row['grand_total']:.2f}")
            non_grocery_splits.append(split)

        # If grocery total is not zero, add it as a split
        if grocery_total != 0:
            grocery_split = create_split(category["grocery"], "", f"{grocery_total:.2f}")
            non_grocery_splits.append(grocery_split)

        # Create the transaction
        transaction = create_transaction(order_date, f"{total_amount:.2f}", "Amazon " + order_id, "", splits=non_grocery_splits)
        transactions.append(transaction)

# Export the transactions to a QIF file by chunks
export_qif_transactions_by_chunk(transactions, 100)


['/content/drive/MyDrive/Colab Notebooks/YNAB//results/transactions_1.qif']